In [1]:
import random


In [20]:
class Environment:
    def __init__(self, size=5):
        self.size = size
        self.reset()

    def reset(self):
        self.agent_position = (random.randint(0, self.size-1), random.randint(0, self.size-1))
        self.passenger_position = (random.randint(0, self.size-1), random.randint(0, self.size-1))
        self.destination_position = (random.randint(0, self.size-1), random.randint(0, self.size-1))
        self.passenger_in_taxi = False
        return self.get_state()

    def get_state(self):
        return self.agent_position, self.passenger_position, self.destination_position, self.passenger_in_taxi

    def step(self, action):
        x, y = self.agent_position

        if action == 'N':
            x = max(0, x - 1)
        elif action == 'S':
            x = min(self.size - 1, x + 1)
        elif action == 'E':
            y = min(self.size - 1, y + 1)
        elif action == 'W':
            y = max(0, y - 1)
        elif action == 'P':
            if self.agent_position == self.passenger_position:
                self.passenger_in_taxi = True
                self.passenger_position = None
        elif action == 'D':
            if self.agent_position == self.destination_position and self.passenger_in_taxi:
                self.passenger_in_taxi = False
                self.destination_position = None

        self.agent_position = (x, y)

        reward = -1  # Default reward is -1 for each step

        if action == 'P' and self.passenger_in_taxi:
            reward += 10  # Picked up the passenger
        elif action == 'D' and self.destination_position is None:
            reward += 20  # Dropped off the passenger
        elif action in ['P', 'D']:
            reward -= 10  # Unsuccessful pick-up or drop-off

        return self.agent_position, reward, self.destination_position is None

class Agent:
    def __init__(self):
        self.actions = ['N', 'S', 'E', 'W', 'P', 'D']

    def choose_action(self, state):
        return random.choice(self.actions)



##### Hier ist eine grundlegende Implementierung eines Q-Learning-Agenten:

- In dieser Implementierung verwendet der Agent die Epsilon-Greedy-Strategie, um die Aktionen auszuwählen. Das bedeutet, dass der Agent mit einer Wahrscheinlichkeit von Epsilon (exploration_rate) eine zufällige Aktion wählt (dies ist der Explorationsteil), und mit einer Wahrscheinlichkeit von 1 - Epsilon die Aktion mit der höchsten erwarteten zukünftigen Belohnung wählt (dies ist der Exploitationsteil).

- Die update_q_value-Methode implementiert die Q-Learning-Update-Regel. Diese Regel sagt, dass die erwartete zukünftige Belohnung für eine Aktion in einem Zustand gleich der aktuellen Belohnung plus der maximalen erwarteten zukünftigen Belohnung im nächsten Zustand ist (abgezinst durch den Discount-Faktor), minus der aktuellen erwarteten zukünftigen Belohnung.

- Der Agent speichert die Q-Werte in einem Wörterbuch, wobei die Schlüssel Tupel aus Zustand und Aktion sind. Der Agent initialisiert die Q-Werte als 0.0, wenn sie noch nicht in der Tabelle sind.

- Der nächste Schritt wäre, den Agenten in der Umgebung zu trainieren. Dies würde bedeuten, den Agenten mehrere Episoden lang in der Umgebung spielen zu lassen und die Q-Werte basierend auf den Belohnungen, die der Agent erhält, zu aktualisieren. Möchten Sie fortfahren?

In [16]:
import numpy as np

class QLearningAgent:
    def __init__(self, actions, learning_rate=0.01, discount_factor=0.99, exploration_rate=0.1):
        self.actions = actions
        self.lr = learning_rate
        self.df = discount_factor
        self.er = exploration_rate
        self.q_table = {}

    def choose_action(self, state):
        if np.random.uniform() < self.er:
            # Explore: choose a random action
            return np.random.choice(self.actions)
        else:
            # Exploit: choose the action with the highest expected future reward
            state = tuple(state)
            q_values = [self.get_q_value(state, action) for action in self.actions]
            return self.actions[np.argmax(q_values)]

    def get_q_value(self, state, action):
        return self.q_table.get((state, action), 0.0)

    def update_q_value(self, state, action, reward, next_state):
        state = tuple(state)
        next_state = tuple(next_state)

        old_q_value = self.get_q_value(state, action)
        max_next_q_value = max([self.get_q_value(next_state, a) for a in self.actions])

        # Q-learning update rule
        new_q_value = old_q_value + self.lr * (reward + self.df * max_next_q_value - old_q_value)
        self.q_table[(state, action)] = new_q_value


In [23]:
env = Environment()
agent = Agent()

In [45]:
state = env.get_state()
print(state)
action = agent.choose_action(state)
print(action)

((3, 4), (0, 1), (1, 1), False)
P


In [46]:
new_state, reward, done = env.step(action)
print('new_state', new_state)
print('reward', reward)
print('done', done)

new_state (3, 4)
reward -11
done False


- In diesem Code trainieren wir den Agenten über 5000 Episoden. In jeder Episode wählt der Agent auf Basis des aktuellen Zustands eine Aktion, führt diese Aktion aus und erhält eine Belohnung und einen neuen Zustand. Dann aktualisiert der Agent die Q-Werte basierend auf der Belohnung und dem neuen Zustand. Der Agent wiederholt diesen Prozess, bis er den Passagier erfolgreich abgesetzt hat, was das Ende der Episode markiert.

- Bitte beachten Sie, dass es mehrere Hyperparameter in diesem Code gibt, die Sie möglicherweise anpassen müssen, um gute Ergebnisse zu erzielen, darunter die Anzahl der Episoden, die Lernrate des Agenten, der Diskontierungsfaktor und die Erkundungsrate.

- Nachdem der Agent trainiert wurde, können Sie ihn testen, indem Sie ihn in der Umgebung agieren lassen und sehen, wie gut er die Aufgabe erfüllt. Sie könnten auch die Q-Werte oder die erhaltene Belohnung im Laufe der Zeit darstellen, um zu sehen, wie der Agent lernt.

In [29]:
# Initialize the agent and the environment
agent = QLearningAgent(['N', 'S', 'E', 'W', 'P', 'D'])
env = Environment()

# Number of episodes to train the agent
n_episodes = 10

for episode in range(n_episodes):
    # Reset the environment and get the initial state
    print(episode)
    state = env.reset()

    while True:
        # The agent chooses an action
        action = agent.choose_action(state)

        # The agent performs the action and gets the reward and new state
        new_state, reward, done = env.step(action)

        # The agent updates the Q-values
        agent.update_q_value(state, action, reward, new_state)

        state = new_state

        if done:
            # The episode ends if the agent has delivered the passenger
            break


0
1
2
3
4
5
6
7
8
9


- In diesem Code trainieren wir den Agenten über 5000 Episoden und verfolgen die Gesamtbelohnung, die der Agent in jeder Episode erhält. Am Ende visualisieren wir die Gesamtbelohnungen pro Episode, um zu sehen, wie sich die Leistung des Agenten im Laufe der Zeit verbessert.

- Dieser Plot kann Ihnen helfen zu beurteilen, ob Ihr Agent lernt: Wenn der Agent richtig lernt, sollten Sie im Allgemeinen eine Verbesserung der Gesamtbelohnungen im Laufe der Zeit sehen, da der Agent lernt, den Passagier effektiver zu transportieren.

- Bitte beachten Sie, dass die Leistung des Agenten von vielen Faktoren abhängt, einschließlich der Lernrate, des Diskontierungsfaktors, der Erkundungsrate und der Anzahl der Trainings-Episoden. Sie müssen möglicherweise mit diesen Parametern experimentieren, um gute Ergebnisse zu erzielen

In [ ]:
import matplotlib.pyplot as plt

# Initialize the agent and the environment
agent = QLearningAgent(['N', 'S', 'E', 'W', 'P', 'D'])
env = Environment()

# Number of episodes to train the agent
n_episodes = 5000
rewards_per_episode = []

for episode in range(n_episodes):
    # Reset the environment and get the initial state
    state = env.reset()
    total_reward = 0

    while True:
        # The agent chooses an action
        action = agent.choose_action(state)

        # The agent performs the action and gets the reward and new state
        new_state, reward, done = env.step(action)

        # The agent updates the Q-values
        agent.update_q_value(state, action, reward, new_state)

        state = new_state
        total_reward += reward

        if done:
            # The episode ends if the agent has delivered the passenger
            break

    rewards_per_episode.append(total_reward)

# Plot the rewards
plt.plot(rewards_per_episode)
plt.title('Total rewards per episode (training)')
plt.xlabel('Episode')
plt.ylabel('Total reward')
plt.show()


In [33]:
# Number of episodes to test the agent
n_test_episodes = 10
test_rewards_per_episode = []

for episode in range(n_test_episodes):
    # Reset the environment and get the initial state
    state = env.reset()
    total_reward = 0
    steps = 0

    while True:
        # The agent chooses an action
        action = agent.choose_action(state)

        # The agent performs the action and gets the reward and new state
        new_state, reward, done = env.step(action)

        state = new_state
        total_reward += reward
        steps += 1

        if done:
            # The episode ends if the agent has delivered the passenger
            break

    print(f"Episode {episode+1}: Total reward = {total_reward}, Steps = {steps}")
    test_rewards_per_episode.append(total_reward)

# Print the average reward per episode
print(f"Average reward: {np.mean(test_rewards_per_episode)}")

# Plot the rewards
plt.plot(test_rewards_per_episode)
plt.title('Total rewards per episode (testing)')
plt.xlabel('Episode')
plt.ylabel('Total reward')
plt.show()
